In [37]:
from models.model import Model
from models.neuralNetworkModel import NeuralNetworkModel
from models.convolutionNeuralNetworkModel import ConvolutionNeuralNetworkModel

import subprocess
import numpy as np
import torch
import os
import shutil

from time import perf_counter

In [26]:
def deleteName(name: str) -> None:
    """Deletes the saved mar file of a specific name"""
    try:
        os.remove(f"model_store/{name}.mar")
        shutil.rmtree("logs")
    except:
        pass

def storeModel(model: Model, name: str, otherConfigs: dict) -> None:
    """
    Needs the saved model with name in model_instances.
    Class of the model should be in models and handler(same name replacing class file name "Model" with "Hanlder")
    in handlers.
    """    

    modelClassName = model.__class__.__name__[0].lower() + model.__class__.__name__[1:]

    ##SAVE CONFIG JSON
    model.saveConfigJson(name, otherConfigs)

    subprocess.Popen(["torch-model-archiver", "--model-name", name,
                      "--version", "1.0",
                      "--model-file", "models/" + modelClassName + ".py",
                      "--serialized-file", "model_instances/" + name + ".pth",
                      "--export-path", "model_store",
                      "--extra-files", f"./models/model.py,./models/{modelClassName}.py,./configs/{name}.json",
                      "--handler", "handlers/" + modelClassName.replace("Model", "Handler") + ".py",
                      ])

In [7]:
import requests
import json

In [ ]:
def performMeasurements(model: Model, name: str) -> list[float]:
    url = f"http://127.0.0.1:8080/predictions/{name}"

    times = []

    for i in [1,5,10,20,50,100]:
        start = perf_counter()
        r = requests.get(url, data=json.dumps({"data": np.random.rand(i, *model.config["shape"])}))
        times.append(perf_counter() - start)

    return times

In [ ]:
modelClassnames = [NeuralNetworkModel, ConvolutionNeuralNetworkModel]
names = ["neuralNetwork", "convolutionNeuralNetwork"]
params = [[(2, [2,2], 2), (10, [10,10], 10), (100,[100,100], 100)], [()]]
extraConfigs = [[{"cuda":False}, {"cuda":True}], [{"cuda":False}, {"cuda":True}]]

for modelClassname, name, paramList, extraConfigList in zip(modelClassnames,names, params, extraConfigs):
    for paramSet in paramList:
        model = modelClassname(paramSet)
        model.save(f"model_instances/{name}.pth")
        for extraConfig in extraConfigList:
            deleteName(name)
            storeModel(model, name, {"cuda":False})
            
            x = input("Press enter when torchserve --start is finished initializing")
            times = performMeasurements(model, name)